## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lon,Max Temp,Wind Speed
0,boly,100,HU,overcast clouds,61,45.9672,18.5183,48.88,1.77
1,hermanus,71,ZA,light rain,66,-34.4187,19.2345,51.91,18.66
2,hohhot,0,CN,clear sky,34,40.8106,111.6522,32.29,4.47
3,beshpagir,9,RU,clear sky,51,45.0217,42.3808,47.62,5.68
4,yambio,100,SS,overcast clouds,98,4.5721,28.3955,67.05,3.98


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like? "))
max_temp = float(input("What is the maximum temperature you would like? "))

What is the minimum temperature you would like? 40
What is the maximum temperature you would like? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lon,Max Temp,Wind Speed
0,boly,100,HU,overcast clouds,61,45.9672,18.5183,48.88,1.77
1,hermanus,71,ZA,light rain,66,-34.4187,19.2345,51.91,18.66
3,beshpagir,9,RU,clear sky,51,45.0217,42.3808,47.62,5.68
4,yambio,100,SS,overcast clouds,98,4.5721,28.3955,67.05,3.98
5,vangaindrano,57,MG,broken clouds,95,-23.3500,47.6000,68.72,5.79


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull()

,City,Cloudiness,Country,Current Description,Humidity,Lat,Lon,Max Temp,Wind Speed
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()
clean_df = preferred_cities_df

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lon"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lon,Hotel Name
0,boly,HU,48.88,overcast clouds,45.9672,18.5183,
1,hermanus,ZA,51.91,light rain,-34.4187,19.2345,
3,beshpagir,RU,47.62,clear sky,45.0217,42.3808,
4,yambio,SS,67.05,overcast clouds,4.5721,28.3955,
5,vangaindrano,MG,68.72,broken clouds,-23.3500,47.6000,
6,khanpur,PK,73.94,clear sky,28.6453,70.6567,
7,rikitea,PF,75.29,overcast clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lon"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    hotel_df.loc[index, "Hotel Name"] = hotels["results"]["name"]
        

TypeError: list indices must be integers or slices, not str

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lon,Hotel Name
0,boly,HU,48.88,overcast clouds,45.9672,18.5183,
1,hermanus,ZA,51.91,light rain,-34.4187,19.2345,
3,beshpagir,RU,47.62,clear sky,45.0217,42.3808,
4,yambio,SS,67.05,overcast clouds,4.5721,28.3955,
5,vangaindrano,MG,68.72,broken clouds,-23.3500,47.6000,
6,khanpur,PK,73.94,clear sky,28.6453,70.6567,
7,rikitea,PF,75.29,overcast clouds,-23.1203,-134.9692,


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lon"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lon"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(20.0, 21.0), zoom_level=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=200, point_radius=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
print(fig)

Figure(layout=FigureLayout(height='420px'))
